In [1]:
!pip install --quiet --upgrade google_cloud_firestore google_cloud_aiplatform langchain langchain-google-vertexai langchain_community langchain_experimental pymupdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 60.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 50.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.4/90.4 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 42.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 59.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.8/131.8 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.9/434.9 kB 28.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 53.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49

In [1]:
import vertexai
from vertexai.language_models import TextEmbeddingModel
from vertexai.generative_models import GenerativeModel

import pickle
from IPython.display import display, Markdown

from langchain_google_vertexai import VertexAIEmbeddings
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_experimental.text_splitter import SemanticChunker

from google.cloud import firestore
from google.cloud.firestore_v1.vector import Vector
from google.cloud.firestore_v1.base_vector_query import DistanceMeasure

In [2]:
# Initialize Vertex AI with your project-id and a location
PROJECT_ID = ! gcloud config get-value project
PROJECT_ID = PROJECT_ID[0]
LOCATION = "us-central1" # @param {type:"string"}
print(PROJECT_ID)
vertexai.init(project=PROJECT_ID, location=LOCATION)

qwiklabs-gcp-01-13a2450742aa


In [3]:
# Populate a variable named embedding_model with an instance of the
# langchain_google_vertexai class VertexAIEmbeddings.
from langchain_google_vertexai import VertexAIEmbeddings
embedding_model = VertexAIEmbeddings(model_name="text-embedding-004")

# Download, process and chunk data semantically

In [6]:
# Download the New York City Department of Health and Mental Hygiene's Food
# Protection Training Manual. This document will serve as our RAG source content.
!gcloud storage cp gs://partner-genai-bucket/genai069/nyc_food_safety_manual.pdf .

Copying gs://partner-genai-bucket/genai069/nyc_food_safety_manual.pdf to file://./nyc_food_safety_manual.pdf

Average throughput: 150.8MiB/s


In [7]:
# https://python.langchain.com/v0.1/docs/modules/data_connection/document_loaders/pdf/#using-pymupdf
# Use the LangChain class PyMuPDFLoader to load the contents of the PDF
from langchain_community.document_loaders import PyMuPDFLoader
loader = PyMuPDFLoader("./nyc_food_safety_manual.pdf")
data = loader.load()

In [8]:
# Create a function to do some basic cleaning on artifacts found in this particular document.
def clean_page(page):
  return page.page_content.replace("-\n","")\
                          .replace("\n"," ")\
                          .replace("\x02","")\
                          .replace("\x03","")\
                          .replace("fo d P R O T E C T I O N  T R A I N I N G  M A N U A L","")\
                          .replace("N E W  Y O R K  C I T Y  D E P A R T M E N T  O F  H E A L T H  &  M E N T A L  H Y G I E N E","")

# Create a variable called cleaned_pages that is a list of strings, with each string being a page of content cleaned by above function.
cleaned_pages = []
for pages in data:
  cleaned_pages.append(clean_page(pages))

In [9]:
# Use LangChain's SemanticChunker with the embedding_model created earlier to split the first five pages of cleaned_pages into text chunks.
# https://python.langchain.com/v0.2/docs/how_to/semantic-chunker/#create-text-splitter
text_splitter = SemanticChunker(embedding_model)
docs = text_splitter.create_documents(cleaned_pages[0:4])
chunked_content = [doc.page_content for doc in docs]

In [10]:
# Use the embedding_model to generate embeddings of the text chunks, saving them to a list called chunked_embeddings.
# To do so, pass your list of chunks to the VertexAIEmbeddings class's embed_documents() method.
# https://python.langchain.com/v0.2/docs/integrations/text_embedding/google_vertex_ai_palm/
chunked_embeddings = embedding_model.embed_documents(chunked_content)

In [11]:
# Above code only chunks & create embeddings of a short section of the
# document for demo purpose. To get the chunks & corresponding embeddings for
# the full document, run the following code to download pre-created chunks
# & embeddings
!gsutil cp gs://partner-genai-bucket/genai069/chunked_content.pkl .
!gsutil cp gs://partner-genai-bucket/genai069/chunked_embeddings.pkl .

chunked_content = pickle.load(open("chunked_content.pkl", "rb"))
chunked_embeddings = pickle.load(open("chunked_embeddings.pkl", "rb"))

Copying gs://partner-genai-bucket/genai069/chunked_content.pkl...
/ [1 files][280.7 KiB/280.7 KiB]                                                
Operation completed over 1 objects/280.7 KiB.                                    
Copying gs://partner-genai-bucket/genai069/chunked_embeddings.pkl...
/ [1 files][  1.8 MiB/  1.8 MiB]                                                
Operation completed over 1 objects/1.8 MiB.                                      


# Prepare your vector database
Create a Firestore database with the default name of (default) in Native Mode and leave the other settings to default.

In [12]:
# Populate a db variable with a Firestore Client.
db = firestore.Client(project="qwiklabs-gcp-01-13a2450742aa")

In [13]:
# Use a variable called collection to create a reference to a collection named food-safety.
collection = db.collection("food-safety")

In [14]:
# Using a combination of our lists chunked_content and chunked_embeddings,
# add a document to your collection for each of your chunked documents.
for i, (content, embedding) in enumerate(zip(chunked_content, chunked_embeddings)):
    doc_ref = collection.document(f"doc_{i}")
    doc_ref.set({
        "content": content,
        "embedding": Vector(embedding)
    })

In [16]:
# Create a vector index for your collection using your embedding field using gcloud firestore indexes command
!gcloud firestore indexes composite create \
--collection-group=food-safety \
--query-scope=COLLECTION \
--field-config field-path=embedding,vector-config='{"dimension":"768", "flat": "{}"}' \
--project="qwiklabs-gcp-01-13a2450742aa"

Create request issued
Created index [CICAgOjXh4EK].


In [17]:
def search_vector_database(query: str):

  context = ""

  # 1. Generate the embedding of the query
  query_embedding = embedding_model.embed_query(query)

  # 2. Get the 5 nearest neighbors from your collection.
  # Call the get() method on the result of your call to
  # find_nearest to retrieve document snapshots.
  vector_query = collection.find_nearest(
    vector_field="embedding",
    query_vector=Vector(query_embedding),
    distance_measure=DistanceMeasure.EUCLIDEAN,
    limit=5,
  )

  # 3. Call to_dict() on each snapshot to load its data.
  # Combine the snapshots into a single string named context
  docs = vector_query.stream()
  context = [result.to_dict()['content'] for result in docs]

  return context

In [18]:
# Call the function with a sample query to confirm it's functionality.
search_vector_database("How should I store food?")

[' Store foods away from dripping condensate , at least six inches above the floor and with enough space between items to encourage air circulation. Freezer Storage Freezing is an excellent method for prolonging the shelf life of foods. By keeping foods frozen solid, the bacterial growth is minimal at best. However, if frozen foods are thawed and then refrozen, then harmful bacteria can reproduce to dangerous levels when thawed for the second time. In addition to that, the quality of the food is also affected. Never refreeze thawed foods, instead use them immediately. Keep the following rules in mind for freezer storage:  Use First In First Out method of stock rotation. All frozen foods should be frozen solid with temperature at 0°F or lower. Always use clean containers that are clearly labeled and marked, and have proper and secure lids. Allow adequate spacing between food containers to allow for proper air circulation. Never use the freezer for cooling hot foods. * * Tip: When receiv

# Deploy a Generative AI application to search your vector store

Now that your vector database is prepared, in this section you will work on the client application to query it and return answers generated by Gemini.

1. Activate Cloud Shell by selecting the the icon that activates cloud shell icon in the upper right of the Cloud Console.

2. To set up the base application and install packages, run the following command in Cloud Shell:
```
gcloud storage cp -r gs://partner-genai-bucket/genai069/gen-ai-assessment .
cd gen-ai-assessment
python3 -m pip install -r requirements.txt
```
3. Run this base Flask application in Cloud Shell with:
```
python3 main.py
```
4. The app will run by default on Port 8080. Preview the app by clicking the Web Preview icon at the top of the Cloud Shell panel, then Preview on port 8080.

5. You should be able to see the app and get a greeting, but if you try to ask FreshBot any questions, it will currently only reply Not implemented.

6. Back in the Cloud Console, you can press CTRL+C to stop the app from running. Click Open Editor at the top of the Cloud Shell panel.

7. Within the Explorer panel on the left of the Cloud Shell Editor, select the file `generative-ai-assessment/main.py`

8. Just below the initialization of the Firestore database client, find the first comment labeled TODO. Assign a reference to your food-safety collection to the collection variable.

Note: As you work through the next few steps, you can test your application by running the command python3 main.py. Alternatively, you can also use the Web Preview option to preview the application.

9. Beneath the next two comments beginning TODO, assign the appropriate models to the variables embedding_model (using the same embedding model version of text-embedding-004 you used earlier in the lab) and gen_model (using the generative model version "gemini-pro" and a temperature of 0).

10. Complete the function search_vector_database as you did above.

11. Complete the ask_gemini function to instruct your gen_model to answer a question based on the context retrieved from the vector database.

Since some food safety content involves knives and potential burns, set Gemini's Dangerous Content safety setting to block only high-probability dangerous content.

12. When you have completed these steps, make sure the Flask app is running in your Cloud Shell Terminal and test it by making sure it can answer the following question:
```
What temperature range do Mesophilic Bacteria grow best in?
```
You should receive a response with a temperature range of 50 to 110 degrees Fahrenheit from the model.

Now, we will deploy the Flask application from this directory to Cloud Run.

13. Navigate in the Cloud Console to Artifact Registry and find the cymbal-repo repo that has been created for you in the us-central1 location. Use the copy button to copy the repo URI.

14. Create a Docker image named cymbal-docker-image using Dockerfile by running the following command (make sure you're in gen-ai-assessment directory):
```
docker build -t cymbal-docker-image -f Dockerfile .
```
15. Push the image to the Artifact Registry repository. Refer to this documentation if you need assistance.
- Find image id using command: `docker images`
- Name the image: `docker tag 9dffbe1841e4  us-central1-docker.pkg.dev/qwiklabs-gcp-04-9a8dc3dcd91f/cymbal-repo/cymbal-image`
- Push the image: `docker push us-central1-docker.pkg.dev/qwiklabs-gcp-04-9a8dc3dcd91f/cymbal-repo/cymbal-image`

16. Deploy the app to Cloud Run as a service named cymbal-freshbot-service. Allow the service to accept unauthenticated invocations.
https://www.youtube.com/watch?v=MM4viHa7k4w 

17. Test your model through its Cloud Run service URL with the following question: At what temperature should dairy products be stored?